In [1]:
import os

In [2]:
%pwd

'/home/meetpatel12_/Projects/6th Sem/Kidney Disease Classification Using MLflow and DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/meetpatel12_/Projects/6th Sem/Kidney Disease Classification Using MLflow and DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/meet1214/kidney-disease-classification.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]="meet1214"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6af09b035f6ae71b885643798dcc7690af76f5a5"

In [6]:
import tensorflow as tf

2025-11-23 20:07:53.601767: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-23 20:07:53.731639: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-23 20:07:53.733399: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-23 20:07:55.796840: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5", compile=False)

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        try:
            is_compiled = getattr(self.model, "_is_compiled", False)
        except Exception:
            is_compiled = False
        if not is_compiled:
            lr = float(self.config.all_params.LEARNING_RATE)
            self.model.compile(
                optimizer=tf.keras.optimizers.SGD(learning_rate=lr),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=["accuracy"]
            )
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        import tensorflow as tf
        if not hasattr(tf.keras, "__version__"):
            import tensorflow as tf_main
            tf.keras.__version__ = tf_main.__version__
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            if tracking_url_type_store != "file":
                try:
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                except Exception as e:
                    from mlflow.exceptions import MlflowException
                    if isinstance(e, MlflowException) and "403" in str(e):
                        print("Registry not permitted, logging model without registration.")
                        mlflow.keras.log_model(self.model, "model")
                    else:
                        raise e
            else:
                mlflow.keras.log_model(self.model, "model")


In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-11-23 20:08:39,955: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-23 20:08:39,960: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-23 20:08:39,961: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 27s 3s/step - loss: 28.8576 - accuracy: 0.4820
[2025-11-23 20:09:08,114: INFO: common: json file saved at: scores.json]


2025/11/23 20:09:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpqsxfk920/model/data/model/assets
[2025-11-23 20:09:11,727: INFO: builder_impl: Assets written to: /tmp/tmpqsxfk920/model/data/model/assets]


2025/11/23 20:09:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpqsxfk920/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.1']. Set logging level to DEBUG to see the full traceback.
/home/meetpatel12_/.conda/envs/kidney/lib/python3.8/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025/11/23 20:09:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


Registry not permitted, logging model without registration.
INFO:tensorflow:Assets written to: /tmp/tmpj0v5o7vk/model/data/model/assets
[2025-11-23 20:09:48,574: INFO: builder_impl: Assets written to: /tmp/tmpj0v5o7vk/model/data/model/assets]


2025/11/23 20:09:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpj0v5o7vk/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.1']. Set logging level to DEBUG to see the full traceback.
